# generate slide window for CNV analysis

In [ ]:
import subprocess
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = 'Arial'
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

# gen slide window bed

In [3]:
chromSizes = pd.read_csv('/mnt/Storage/home/wangyiman/source/bySpecies/danRer11_2/chromSizes/danRer11_2_main.chrom.sizes', 
                         sep = "\t", header = None, names = ['chr', 'length'])
chromSizes.index = chromSizes['chr']
chromSizes = chromSizes.drop('chr', axis = 1).to_dict()['length']
chromSizes

{'chr1': 59578282,
 'chr10': 45420867,
 'chr11': 45484837,
 'chr12': 49182954,
 'chr13': 52186027,
 'chr14': 52660232,
 'chr15': 48040578,
 'chr16': 55266484,
 'chr17': 53461100,
 'chr18': 51023478,
 'chr19': 48449771,
 'chr2': 59640629,
 'chr20': 55201332,
 'chr21': 45934066,
 'chr22': 39133080,
 'chr23': 46223584,
 'chr24': 42172926,
 'chr25': 37502051,
 'chr3': 62628489,
 'chr4': 74498832,
 'chr5': 72500376,
 'chr6': 60270059,
 'chr7': 74282399,
 'chr8': 54304671,
 'chr9': 56459846,
 'chrM': 16596}

In [4]:
slide_window_df = pd.DataFrame()
for chrom in list(chromSizes.keys()) :
    start = list(range(0, chromSizes[chrom] - 200000 + 10000, 10000))
    end = list(range(200000, chromSizes[chrom], 10000)) + [chromSizes[chrom]]
    df = pd.DataFrame()
    df['start'] = start
    df['end'] = end
    df['chr'] = chrom
    slide_window_df = pd.concat([slide_window_df, df])
    
slide_window_df = slide_window_df.reset_index()
slide_window_df['bin_name'] = 'bin_' + slide_window_df['chr'] + '_' + slide_window_df['index'].astype(str)
slide_window_df = slide_window_df[['chr', 'start', 'end', 'bin_name']]
slide_window_df = slide_window_df.fillna(0)
slide_window_df['start'] = slide_window_df['start'].astype(int)
slide_window_df

,chr,start,end,bin_name
0,chr1,0,200000,bin_chr1_0
1,chr1,10000,210000,bin_chr1_1
2,chr1,20000,220000,bin_chr1_2
3,chr1,30000,230000,bin_chr1_3
4,chr1,40000,240000,bin_chr1_4
...,...,...,...,...
133687,chr9,56230000,56430000,bin_chr9_5623
133688,chr9,56240000,56440000,bin_chr9_5624
133689,chr9,56250000,56450000,bin_chr9_5625
133690,chr9,56260000,56459846,bin_chr9_5626


In [5]:
slide_window_df.to_csv('/mnt/Storage/home/wangyiman/source/bySpecies/danRer11_2/genome_bin/danRer11_2_main.200kWin.10kStep.bed', header = False, index = False, sep = "\t")


# window coverage

In [ ]:
%%bash
cd ~/maternal_loading/1.private_data/20221209_WGS/analysis
slide_window=/mnt/Storage/home/wangyiman/source/bySpecies/danRer11_2/genome_bin/danRer11_2_main.200kWin.10kStep.bed
for bdg in $(ls ../2_signal/*.bedGraph);do
    name1=$(basename $bdg)
    name=${name1%%.bedGraph}
    bedtools intersect -b $bdg -a $slide_window -wa -wb -sorted > ${name}.intersect_SlideWin.bed
done